Image based approach

Dataset

In [1]:
import os
import time
import numpy as np
import face_recognition
from skimage.transform import resize
from skimage.util import pad

%matplotlib inline
import matplotlib.pyplot as plt


LABELS = ['22q11','Angelman','Apert','CDL','Down','FragileX','Marfan','Progeria','Sotos','TreacherCollins','Turner','Williams']
IMAGE_PATCH_SIZE = 256
EXTRA_PAD = 32

#LABELS.remove('CDL')


def load_dataset_patches(setPath):
    X = []
    Y = []

    for label in LABELS:
            
        directory = os.path.join(setPath,label)
        for f in os.listdir(directory):
            path = os.path.join(directory,f)
            
            if os.path.isfile(path) and f != 'desktop.ini':
                
                # Extract landmarks
                image = face_recognition.load_image_file(path)
                face_landmarks_list = face_recognition.face_landmarks(image)

                # Process faces
                for landmarks in face_landmarks_list:
                    
                    # find dimensions
                    top = image.shape[0]
                    bottom = 0
                    left = image.shape[1]
                    right = 0
                    
                    for featureType in landmarks.keys():
                        for point in landmarks[featureType]:
                            (x,y) = point
                            
                            if x < left:
                                left = x
                            if x > right:
                                right = x
                            if y < top:
                                top = y
                            if y > bottom:
                                bottom = y
                                
                    top = top - EXTRA_PAD if top - EXTRA_PAD >= 0 else 0
                    bottom = bottom + EXTRA_PAD if bottom + EXTRA_PAD < image.shape[0] else image.shape[0] - 1
                    left = left - EXTRA_PAD if left - EXTRA_PAD >= 0 else 0
                    right = right + EXTRA_PAD if right + EXTRA_PAD < image.shape[1] else image.shape[1] - 1
                    
                    height = bottom - top
                    width = right - left
                    
                    if height > width:
                        # fix height
                        diff = height - width
                        bottom = bottom - diff
                        
                    elif height < width:
                        # fix width
                        diff = width - height
                        right = right - diff
                            
                    
                    # extract patch from image
                    img_patch = image[top:bottom,left:right,:]
                    img_patch_resized = resize(img_patch,(IMAGE_PATCH_SIZE,IMAGE_PATCH_SIZE,3))

                    X.append(img_patch_resized)
                    Y.append(label)
                    
    # pack into numpy structure
    X_pack = np.zeros([len(X),IMAGE_PATCH_SIZE,IMAGE_PATCH_SIZE,3])
    for i in range(0,len(X)):
        X_pack[i,:,:] = X[i]
    Y_pack = np.array(Y)
    
    return X_pack, Y_pack

In [2]:
X_test, y_test = load_dataset_patches('./data/raw/Test')
print(X_test.shape)
print(y_test.shape)

(312, 256, 256, 3)
(312,)


In [3]:
X_val, y_val = load_dataset_patches('./data/raw/Validate')
print(X_val.shape)
print(y_val.shape)

(281, 256, 256, 3)
(281,)


In [4]:
X_train, y_train = load_dataset_patches('./data/raw/Train')
print(X_train.shape)
print(y_train.shape)

(1023, 256, 256, 3)
(1023,)


In [5]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(y_train)
Y_test = encoder.transform(y_test)
Y_val = encoder.transform(y_val)

print(Y_train.shape)
print(Y_test.shape)

(1023, 12)
(312, 12)


Model

In [6]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

model = Sequential()
model.add(Conv2D(64, (7,7), activation='relu', input_shape=(IMAGE_PATCH_SIZE,IMAGE_PATCH_SIZE,3)))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(Conv2D(32, (5,5), activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Using TensorFlow backend.
/home/curt/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/curt/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/curt/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/curt/.local/lib/python3.6/site-packages/tensorflow/python/framewo

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 250, 250, 64)      9472      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 246, 246, 64)      102464    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 242, 242, 32)      51232     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 240, 240, 32)      9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 238, 238, 32)      9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1812608)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

Train

In [ ]:
history = model.fit(X_train, Y_train, batch_size=8, epochs=15, shuffle=True, validation_data=(X_val,Y_val))
print(history)


Train on 1023 samples, validate on 281 samples
Epoch 1/15
   8/1023 [..............................] - ETA: 31:01 - loss: 2.4556 - accuracy: 0.1250

Evaluate

In [ ]:
score = model.evaluate(X_train, Y_train, batch_size=32, verbose=1)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

Y_pred = model.predict(X_train)
y_pred = encoder.inverse_transform(Y_pred)

conf = confusion_matrix(y_train, y_pred)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(conf, index = [i for i in LABELS],
                  columns = [i for i in LABELS])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
score = model.evaluate(X_val, Y_val, batch_size=32, verbose=1)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
score = model.evaluate(X_test, Y_test, batch_size=32, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

Y_pred = model.predict(X_test)
y_pred = encoder.inverse_transform(Y_pred)

conf = confusion_matrix(y_test, y_pred)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(conf, index = [i for i in LABELS],
                  columns = [i for i in LABELS])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

Save

In [ ]:
import time

save_dir = './models'
model_name = 'image-classifier-{}.h5'.format(int(time.time()))

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save_weights(model_path)
print('Saved trained model at %s ' % model_path)